# CREACIÓN DEL DATAFRAME

In [1]:
from calendar import monthrange
import os.path
import numpy as np
import pandas as pd
from datetime import datetime, date
import xlrd 
from openpyxl import load_workbook

In [2]:
MESES = {
        1: 'Enero',
        2: 'Febrero',
        3: 'Marzo',
        4: 'Abril',
        5: 'Mayo',
        6: 'Junio',
        7: 'Julio',
        8: 'Agosto',
        9: 'Septiembre',
        10: 'Octubre',
        11: 'Noviembre',
        12: 'Diciembre'
        }

In [3]:
HEADERS = [ ('FECHA', ''),
            ('HORA', ''),
            ('CAUDAL','Entr l/s'),
            ('Cal 1ria', 'Kg.'),
            ('Cal 1ria', 'Dosis'),
            ('AGUA CRUDA', 'p.H'),
            ('AGUA CRUDA', 'Color'),
            ('AGUA CRUDA', 'NTU'),
            ('AGUA CRUDA', 'Alcalinidad'),
            ('AGUA CRUDA', 'Conductividad'),
            ('COAGULANTE', 'Granulado'),
            ('COAGULANTE', 'Liquido'),
            ('COAGULANTE', 'Dosis'),
            ('AGUA TRATADA', 'Cloro'),
            ('AGUA TRATADA', 'Alcalinidad'),
            ('AGUA TRATADA', 'p.H'),
            ('AGUA TRATADA', 'Color'),
            ('AGUA TRATADA', 'NTU')
          ]
header = pd.MultiIndex.from_tuples(HEADERS)

In [4]:
df = pd.DataFrame()
df_total = pd.DataFrame()
años = list(range(2013,2023))
meses = list(range(1,13))
horas = list(range(25))

In [5]:
def numero_a_mes(numero):
    if 1 <= numero <= 12:
        return MESES[numero]
    else:
        raise ValueError("El número debe estar entre 1 y 12.")

In [6]:
def detectar_fila_cabecera(archivo_excel):
    wb = load_workbook(filename=archivo_excel, read_only=True)
    filas_cabecera = {}
    palabra_clave = 'HO'

    for hoja in wb:
        fila_cabecera = None
        for index, row in enumerate(hoja.iter_rows(), start=1):
            if any(palabra_clave in str(cell.value) for cell in row if cell.value is not None):
                fila_cabecera = index
                break
        filas_cabecera[hoja.title] = fila_cabecera

    return filas_cabecera

In [7]:
def dataframe_mes(fila, mes, days, año, archivo):
    for i in list(range(days)):
        x = f'{i+1}'
        fecha = f'{i+1}'+'/'+f'{mes}'+'/'+f'{año}'
        if i == 0:
            df_empty = pd.DataFrame(columns=HEADERS)
            df = pd.read_excel(archivo, sheet_name=x, header=[0, 1], skiprows=fila[x] - 1, nrows=25)
            common_columns = df_empty.columns.intersection(df.columns)
            df_common = df[common_columns]
            df = pd.concat([df_empty, df_common], axis=0, ignore_index=True)
            df[('FECHA', '')] = fecha
            df[('HORA', '')] = horas
        else:
            df_read = pd.read_excel(archivo, sheet_name=x, header=[0, 1], skiprows=fila[x] - 1, nrows=25)
            df_read[('FECHA', '')] = fecha
            df_read[('HORA', '')] = horas
            df = pd.concat([df, df_read])

    return df

In [8]:
def crear_dataframe():

    for año in años:
        for mes in meses:
            CURR_NUM_DAYS = monthrange(año, mes)[1]
            mes_name = numero_a_mes(mes)
            archivo = f"./data/{año}/{mes_name}{año}.xlsx"
            print(archivo)

            if os.path.exists(archivo):
                fila = detectar_fila_cabecera(archivo)
                df_mes = dataframe_mes(fila, mes, CURR_NUM_DAYS, año, archivo)
            else:
                print(f"Archivo no encontrado: {archivo}. Creando datos NaN para el mes {mes} del año {año}.")
                if mes == 12:
                    año = año+1
                    mes = 1
                else:
                    mes = mes+1

            if año == 2013 and mes == 1:
                df_total = df_mes
            else:
                df_total = pd.concat([df_total, df_mes])

    return df_total[HEADERS].reset_index(drop=True)

In [9]:
dataframe = crear_dataframe()

./data/2013/Enero2013.xlsx
./data/2013/Febrero2013.xlsx
./data/2013/Marzo2013.xlsx
./data/2013/Abril2013.xlsx
./data/2013/Mayo2013.xlsx
./data/2013/Junio2013.xlsx
./data/2013/Julio2013.xlsx
./data/2013/Agosto2013.xlsx
./data/2013/Septiembre2013.xlsx
./data/2013/Octubre2013.xlsx
./data/2013/Noviembre2013.xlsx
./data/2013/Diciembre2013.xlsx
./data/2014/Enero2014.xlsx
./data/2014/Febrero2014.xlsx
./data/2014/Marzo2014.xlsx
./data/2014/Abril2014.xlsx
./data/2014/Mayo2014.xlsx
./data/2014/Junio2014.xlsx
./data/2014/Julio2014.xlsx
./data/2014/Agosto2014.xlsx
./data/2014/Septiembre2014.xlsx
./data/2014/Octubre2014.xlsx
./data/2014/Noviembre2014.xlsx
./data/2014/Diciembre2014.xlsx
./data/2015/Enero2015.xlsx
./data/2015/Febrero2015.xlsx
./data/2015/Marzo2015.xlsx
./data/2015/Abril2015.xlsx
./data/2015/Mayo2015.xlsx
./data/2015/Junio2015.xlsx
./data/2015/Julio2015.xlsx
./data/2015/Agosto2015.xlsx
./data/2015/Septiembre2015.xlsx
./data/2015/Octubre2015.xlsx
./data/2015/Noviembre2015.xlsx
./data/2

In [10]:
dataframe

FECHA HORA   CAUDAL Cal 1ria       AGUA CRUDA                \
                       Entr l/s      Kg. Dosis        p.H  Color    NTU   
0        1/1/2013    0      NaN      NaN   NaN        NaN    NaN    NaN   
1        1/1/2013    1    545.0      NaN   NaN        7.5    6.5    2.1   
2        1/1/2013    2    538.0      NaN   NaN        NaN    NaN    NaN   
3        1/1/2013    3    516.0      NaN   NaN        7.8    6.0    2.4   
4        1/1/2013    4    526.0      NaN   NaN        NaN    NaN    NaN   
...           ...  ...      ...      ...   ...        ...    ...    ...   
91270  31/12/2022   20    600.0      NaN   NaN        7.4  425.0  657.0   
91271  31/12/2022   21    620.0      NaN   NaN        7.5  321.0  404.0   
91272  31/12/2022   22    620.0      NaN   NaN        7.3  322.0  402.0   
91273  31/12/2022   23    510.0      NaN   NaN        7.4  300.0  390.0   
91274  31/12/2022   24    500.0      NaN   NaN        6.9  110.0   73.4   

                                COAGULANTE               AGUA TRATADA  \
      Alcalinidad Conductividad  Granulado Liquido Dosis        Cloro   
0             NaN           NaN        NaN     NaN   NaN          NaN   
1             NaN           NaN        NaN     NaN   NaN          1.2   
2             NaN           NaN        NaN     NaN   NaN          NaN   
3             NaN           NaN        NaN     NaN   NaN          1.2   
4             NaN           NaN        NaN     NaN   NaN          NaN   
...           ...           ...        ...     ...   ...          ...   
91270        25.0          55.5        NaN     NaN  40.0          1.5   
91271        25.0          55.7        NaN     NaN  40.0          NaN   
91272        25.0          55.4        NaN     NaN  40.0          1.6   
91273        25.0          55.5      650.0     NaN  40.0          1.5   
91274        28.0          58.1        NaN     NaN  40.0          1.5   

                                   
      Alcalinidad  p.H Color  NTU  
0             NaN  NaN   NaN  NaN  
1             NaN  6.8   2.8  0.7  
2             NaN  NaN   NaN  NaN  
3             NaN  6.9   3.0  0.6  
4             NaN  NaN   NaN  NaN  
...           ...  ...   ...  ...  
91270        17.0  6.6   2.0  1.1  
91271        17.0  6.5   2.0  1.1  
91272        13.0  6.6   2.0  1.4  
91273        14.0  6.8   2.0  1.2  
91274        17.0  6.9   2.0  1.2  

[91275 rows x 18 columns]

In [11]:
dataframe.to_csv('./data/dataframe.csv', sep=',')

In [12]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91275 entries, 0 to 91274
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   (FECHA, )                    91275 non-null  object
 1   (HORA, )                     91275 non-null  int64 
 2   (CAUDAL, Entr l/s)           88463 non-null  object
 3   (Cal 1ria, Kg.)              357 non-null    object
 4   (Cal 1ria, Dosis)            906 non-null    object
 5   (AGUA CRUDA, p.H)            87642 non-null  object
 6   (AGUA CRUDA, Color)          82226 non-null  object
 7   (AGUA CRUDA, NTU)            88103 non-null  object
 8   (AGUA CRUDA, Alcalinidad)    69110 non-null  object
 9   (AGUA CRUDA, Conductividad)  60681 non-null  object
 10  (COAGULANTE, Granulado)      6326 non-null   object
 11  (COAGULANTE, Liquido)        564 non-null    object
 12  (COAGULANTE, Dosis)          45279 non-null  object
 13  (AGUA TRATADA, Cloro)        79